In [1]:
# Load the generative model

include("generative_model.jl")

DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Any], false, Union{Nothing, Some{Any}}[nothing], var"##generate_table_world#312", Bool[0], false)

In [2]:
function condition(model, args, list_of_conditions)
    
    all_satisfied = false
    
    world = model(args)
    counter = 0
    while all_satisfied == false
#         function apply(f, w)
#             return f(w)
#         end
        if all([c(world) for c in list_of_conditions])
#         if all(map(f -> f(world), list_of_conditions))
            all_satisfied = true
        else
            world = model(args)
        end
        counter += 1
    end
    
    return world, ("NUM_SIMS", counter)
            
end

condition (generic function with 1 method)

In [3]:
default_world_dims = [1000, 1000, 1000]

# scene = generate_table_world(default_world_dims)
# println(scene)

# The table is extremely large
cond1 = w -> (w[1]["dims"][1] > 745) && (w[1]["dims"][3] > 745)

# The table is extremely tall

cond2 = w -> w[1]["dims"][2] > 745

# There are 9 blocks
cond3 = w -> length(w) == 9 + 1

# There are at least 5 red blocks

@time condition(generate_table_world, default_world_dims, [cond1, cond2, cond3])

  2.202933 seconds (26.27 M allocations: 1.582 GiB, 5.67% gc time, 22.27% compilation time)


(Any[Dict{String, Any}("mass" => 100, "base_shape" => ["table"], "abs_spatial" => Dict("x" => 500.0, "z" => 500.0, "y" => 373.5), "dims" => [746, 747, 746], "color" => "white", "object_type" => "table"), Dict{String, Any}("volume" => 1728, "color" => "yellow", "base_shape" => ["cube"], "abs_spatial" => Dict("x" => 840.0378634006739, "z" => 870.965429334162, "y" => 753.0), "dims" => [12, 12, 12], "mass" => 2.16, "object_type" => "block"), Dict{String, Any}("volume" => 1000, "color" => "red", "base_shape" => ["cube"], "abs_spatial" => Dict("x" => 133.15015300892694, "z" => 204.52153667108493, "y" => 752.0), "dims" => [10, 10, 10], "mass" => 1.25, "object_type" => "block"), Dict{String, Any}("volume" => 3375, "color" => "red", "base_shape" => ["cube"], "abs_spatial" => Dict("x" => 325.14524586085975, "z" => 351.84245930409685, "y" => 754.5), "dims" => [15, 15, 15], "mass" => 4.21875, "object_type" => "block"), Dict{String, Any}("volume" => 8000, "color" => "red", "base_shape" => ["cube"],

In [4]:
# How many blocks are there?

# How many blocks are on the table?

# How many red blocks are on the table?

# Are there more yellow blocks or red blocks?

# What is the color of the heaviest red block?